In [103]:
import numpy as np
import tensorflow as tf
import keras
from datetime import date

for g in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(g, True)

print(tf.config.list_physical_devices())

np.random.seed(42)
tf.random.set_seed(42)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


 9. Train an Encoder–Decoder model that can convert a date string from one format
 to another (e.g., from “April 22, 2019” to “2019-04-22”).

In [104]:
# cannot use strftime()'s %B format since it depends on the locale
MONTHS = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]

def random_dates(n_dates):
    min_date = date(1000, 1, 1).toordinal()
    max_date = date(9999, 12, 31).toordinal()

    ordinals = np.random.randint(max_date - min_date, size=n_dates) + min_date
    dates = [date.fromordinal(ordinal) for ordinal in ordinals]

    x = [MONTHS[dt.month - 1] + " " + dt.strftime("%d, %Y") for dt in dates]
    y = [dt.isoformat() for dt in dates]
    return x, y

In [105]:
n_dates = 3
x_example, y_example = random_dates(n_dates)
print("{:25s}{:25s}".format("Input", "Target"))
print("-" * 50)
for idx in range(n_dates):
    print("{:25s}{:25s}".format(x_example[idx], y_example[idx]))

Input                    Target                   
--------------------------------------------------
September 20, 7075       7075-09-20               
May 15, 8579             8579-05-15               
January 11, 7103         7103-01-11               


In [106]:
INPUT_CHARS = "".join(sorted(set("".join(MONTHS) + "0123456789, ")))
INPUT_CHARS

' ,0123456789ADFJMNOSabceghilmnoprstuvy'

In [107]:
OUTPUT_CHARS = "0123456789-"
SPECIAL_TOKENS = ["<pad>", "<sos>", "<eos>"]

pad_token = SPECIAL_TOKENS[0]
start_token = SPECIAL_TOKENS[1]
end_token = SPECIAL_TOKENS[2]

In [108]:
encoder_tokens = tf.constant(SPECIAL_TOKENS + list(INPUT_CHARS))
encoder_ids    = tf.range(tf.shape(encoder_tokens)[0], dtype=tf.int64)

encoder_initializer = tf.lookup.KeyValueTensorInitializer(encoder_tokens, encoder_ids)
encoder_table = tf.lookup.StaticHashTable(encoder_initializer, default_value=-1)  # choose any sentinel

In [109]:
encoder_table.lookup(tf.constant("<eos>", dtype=tf.string))

<tf.Tensor: shape=(), dtype=int64, numpy=2>

In [110]:
decoder_tokens = tf.constant(SPECIAL_TOKENS + list(OUTPUT_CHARS))
decoder_ids    = tf.range(tf.shape(decoder_tokens)[0], dtype=tf.int64)

decoder_initializer = tf.lookup.KeyValueTensorInitializer(decoder_tokens, decoder_ids)
decoder_table = tf.lookup.StaticHashTable(decoder_initializer, default_value=-1)  # choose any sdetinel

In [111]:
decoder_table.lookup(tf.constant("<eos>", dtype=tf.string))

<tf.Tensor: shape=(), dtype=int64, numpy=2>

In [112]:
vocab_size = len(SPECIAL_TOKENS) + len(INPUT_CHARS)

In [113]:
# ...existing code...
def vectorize_and_pad(batch, table, add_sos_eos=None):
    chars = tf.strings.unicode_split(batch, "UTF-8")  # RaggedTensor: [batch, seq]
    batch_size = tf.shape(chars)[0]

    if add_sos_eos == "both":
        sos = tf.fill([batch_size, 1], start_token)
        eos = tf.fill([batch_size, 1], end_token)  
        chars = tf.concat([sos, chars, eos], axis=1)
    elif add_sos_eos == "sos":
        sos = tf.fill([batch_size, 1], start_token)
        chars = tf.concat([sos, chars], axis=1)
    elif add_sos_eos == "eos":
        eos = tf.fill([batch_size, 1], end_token)
        chars = tf.concat([chars, eos], axis=1)

    ids = table.lookup(chars)
    padded_batch = ids.to_tensor(default_value=0)
    return tf.cast(padded_batch, tf.int32)

In [114]:
src_dates, trg_dates = random_dates(10000)
encoder_inputs  = vectorize_and_pad(src_dates, encoder_table)
decoder_inputs = vectorize_and_pad(trg_dates, decoder_table, add_sos_eos="both")
decoder_targets = decoder_inputs[:, 1:]  # shift for teacher forcing
decoder_inputs = decoder_inputs[:, :-1]

In [115]:
val_src_dates, val_trg_dates = random_dates(2000)
val_encoder_inputs  = vectorize_and_pad(val_src_dates, encoder_table)
val_decoder_inputs = vectorize_and_pad(val_trg_dates, decoder_table, add_sos_eos="both")
val_decoder_targets = val_decoder_inputs[:, 1:]  # shift for teacher forcing
val_decoder_inputs = val_decoder_inputs[:, :-1]

In [116]:
decoder_vocab_size = len(SPECIAL_TOKENS) + len(OUTPUT_CHARS)

from keras import layers, Model

# Encoder
enc_inputs = layers.Input(shape=(None,), dtype='int32', name='encoder_tokens')
enc_emb = layers.Embedding(vocab_size, 15)(enc_inputs)
enc_out, enc_state = layers.GRU(64, return_sequences=True, return_state=True)(enc_emb)

# Decoder
dec_inputs = layers.Input(shape=(None,), dtype='int32', name='decoder_tokens')
dec_emb = layers.Embedding(decoder_vocab_size, 15)(dec_inputs)
dec_gru = layers.GRU(64, return_sequences=True)
dec_out = dec_gru(dec_emb, initial_state=enc_state)

# Attention
attention = layers.Attention()([dec_out, enc_out])
concat = layers.Concatenate()([dec_out, attention])

# Output
outputs = layers.Dense(decoder_vocab_size, activation='softmax')(concat)

# Model
model = Model([enc_inputs, dec_inputs], outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [117]:
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_tokens      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_tokens      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_10        │ (None, None, 15)  │        615 │ encoder_tokens[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_11        │ (None, None, 15)  │        210 │ decoder_tokens[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_10 (GRU)        │ [(None, None,     │     15,552 │ embedding_10[0][… │
│                     │ 64), (None, 64)]  │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_11 (GRU)        │ (None, None, 64)  │     15,552 │ embedding_11[0][… │
│                     │                   │            │ gru_10[0][1]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_5         │ (None, None, 64)  │          0 │ gru_11[0][0],     │
│ (Attention)         │                   │            │ gru_10[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, None, 128) │          0 │ gru_11[0][0],     │
│ (Concatenate)       │                   │            │ attention_5[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, None, 14)  │      1,806 │ concatenate_5[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 33,735 (131.78 KB)

 Trainable params: 33,735 (131.78 KB)

 Non-trainable params: 0 (0.00 B)

In [118]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
check_cb = ModelCheckpoint("best_date_encoder_decoder.keras", save_best_only=True, monitor="loss")
early_stopping_cb = EarlyStopping(patience=20, min_delta=1e-3)

history = model.fit(
    {"encoder_tokens": encoder_inputs, "decoder_tokens": decoder_inputs},
    decoder_targets,
    epochs=100,
    validation_data=({"encoder_tokens": val_encoder_inputs, "decoder_tokens": val_decoder_inputs}, val_decoder_targets),
    callbacks=[check_cb, early_stopping_cb]
)

Epoch 1/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.4852 - loss: 1.4972 - val_accuracy: 0.8519 - val_loss: 0.6025
Epoch 2/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.9824 - loss: 0.1218 - val_accuracy: 0.9999 - val_loss: 0.0196
Epoch 3/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 1.0000 - loss: 0.0100 - val_accuracy: 0.9999 - val_loss: 0.0053
Epoch 4/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 1.0000 - loss: 0.0035 - val_accuracy: 0.9999 - val_loss: 0.0028
Epoch 5/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 1.0000 - loss: 0.0019 - val_accuracy: 0.9999 - val_loss: 0.0018
Epoch 6/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 1.0000 - loss: 0.0012 - val_accuracy: 0.9999 - val_loss: 0.0012
Epoch 7/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 1.0000 - loss: 8.4041e-04 - val_accuracy: 0.9999 - val_loss: 9.4130e-04
Epoch 8/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 1.0000 - loss: 6